# MONTE CARLO SEQUENCIAL

## IMPORTAÇÃO DE BIBLIOTECAS

In [14]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

## LEITURA DE DADOS

In [5]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']


In [6]:
#Deixando os dados em horas e nos formatos conhecidos
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

In [8]:
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760
D_LIN['MTTF'] = 1 / D_LIN['Lambda']
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

In [9]:
Ng = len(D_GEN)
geradores = np.arange(1,Ng+1)
Nl = len(D_LIN)
linhas = np.arange(1,Nl+1)
Nc = len(D_LOAD)
cargas = np.arange(1,Nc+1)

In [12]:
#Funções Auxiliares:
def sortear_tempo_falha(mttf):
    return np.random.exponential(mttf)

def sortear_tempo_reparo(mttr):
    return np.random.exponential(mttr)

def inicializar_componentes(D_GEN, D_LIN):
    # Inicializar componentes, 1=operando, 0=falha
    estados_ger = np.ones(len(D_GEN))
    estados_lin = np.ones(len(D_LIN))

    #Sortear o tempo de falha para cada componente
    tempos_gen = [sortear_tempo_falha(mttf) for mttf in D_GEN['MTTF']]
    tempos_lin = [sortear_tempo_falha(mttf) for mttf in D_LIN['MTTF']]

    return estados_ger, estados_lin, tempos_gen, tempos_lin


In [13]:
#Parâmetros principais:
tol = 0.01 #Tolerância para  o critério de parada
NY_MAX = 100 #Máximo número de anos para simular
NY = 0 #Contador de anos

#Inicializando Índices
F_LOLE = 0
F_EENS = 0
F_LOLF = 0
F_LOLD = 0

In [ ]:
#Algoritmo de Monte Carlo Sequencial:
while NY < NY_MAX:
    NY = NY + 1
    TSIS = 0 #Inicializa o tempo acumulado do sistema

    #Inicializa os estados e tempos de residência:
    estados_gen, estados_lin, tempos_gen, tempos_lin = inicializar_componentes(D_GEN, D_LIN)
    TG = tempos_gen + tempos_lin  # Tempos globais de residência (falha ou reparo)

    while TSIS < 8760:
        # Determina o componente com o menor tempo de residência (falha ou recuperação)
        TGi = min(TG)
        if TGi > 8760:
            TGi = 8760
        
        TDUR = TGi - TSIS # Tempo de duração do estado atual
        TSIS = TGi # Atualiza o tempo acumulado do sistema

        #Atualiza o impacto do estado atual nos índies de confiabilidade
        z_ger = estados_gen
        z_lin =  estados_lin

        #Calcula o FPO
        carga_cortada = calcular_fpo(z_ger, z_lin)

        if carga_cortada > 0:
            F_LOLE += TDUR #Tempo de perda de carga (LOLE)
            F_EENS += carga_cortada * TDUR #Energia não fornecida (EENS)
            F_LOLF += 1 #Frequência de corte de carga (LOLF)
        
        



In [ ]:
####Definir os Z's de acordo com o problema!